# Running a Qiskit QAOA circuit with Quasar/Vulcan

In this notebook, we will show how to take a QAOA circuit written in Qiskit, transform it into a Quasar circuit with *qusetta*, and then run it with various Quasar features. If you want to install *qusetta* in a virtual enviroment and then add your virtual enviroment to your list of jupyter kernels, see [this post](https://janakiev.com/blog/jupyter-virtual-envs/) for more details.

**Contents**
1. [The problem](#1.-The-problem)
2. [The circuits](#2.-The-circuits)
3. [The cost function](#3.-The-cost-function)
4. [Getting the cover](#4.-Getting-the-cover)
5. [Simulating the circuits](#5.-Simulating-the-circuits)
6. [Running and timing the circuits](#6.-Running-and-timing-the-circuits)
7. [Outlook](#7.-Outlook)
---

## 1. The problem

For this example, we will work with the Vertex Cover problem. We will follow the procedure in this [qiskit example notebook](https://github.com/Qiskit/qiskit-community-tutorials/blob/master/optimization/vertex_cover.ipynb) to create the problem and the QAOA circuit in qiskit.

We'll begin exactly as they begin -- by creating a random graph (and seeding random for reproducability of the notebook).

In [1]:
import numpy as np
from qiskit.optimization.applications.ising.common import random_graph

np.random.seed(123)
num_nodes = 22
w = random_graph(num_nodes, edge_prob=0.8, weight_range=10)

Next we'll create the qubit operator and the corresponding QAOA instance. We'll fix a depth $p$ to work with throughout this notebook.

In [2]:
from qiskit.optimization.applications.ising import vertex_cover
from qiskit.aqua.algorithms import QAOA

qubit_op, offset = vertex_cover.get_operator(w)

p = 10
qaoa = QAOA(qubit_op, p=p)

## 2. The circuits

Now we'll write a function that takes in $\beta_1, \dots, \beta_p$, and $\gamma_1, \dots, \gamma_p$ and outputs the corrresponding qiskit QAOA circuit. Note that `params` is a list such that `params[:p]` is $[\gamma_1, \dots, \gamma_p]$ and `params[p:]` is $[\beta_1, \dots, \beta_p]$.

In [3]:
import qiskit
from typing import List

def create_qiskit_circuit(params: List[float]) -> qiskit.QuantumCircuit:
    assert len(params) == 2 * p, "invalid number of angles"
    return qaoa.var_form.construct_circuit(params)

Next we'll write a function that uses *qusetta* to convert the qiskit circuit to a cirq circuit.

In [4]:
import qusetta as qs
import cirq

def create_cirq_circuit(params: List[float]) -> cirq.Circuit:
    qiskit_circuit = create_qiskit_circuit(params)
    return qs.Qiskit.to_cirq(qiskit_circuit)

Next we'll write a function that uses *qusetta* to convert the qiskit circuit to a quasar circuit.

In [5]:
import quasar

def create_quasar_circuit(params: List[float]) -> quasar.Circuit:
    qiskit_circuit = create_qiskit_circuit(params)
    return qs.Qiskit.to_quasar(qiskit_circuit)

Let's see how big the circuit is.

In [6]:
c = create_quasar_circuit([0.] * (2*p))
print("Number of qubits :", c.nqubit)
print("Number of gates  :", c.ngate)

Number of qubits : 22
Number of gates  : 6102


## 3. The cost function

The cost function of the circuit is the expectation value of the qubit operator.

In [7]:
def expectation_value(statevector: np.ndarray) -> float:
    # note that the second element (eg [1]) is the standard deviation
    return offset + qubit_op.evaluate_with_statevector(statevector)[0].real

## 4. Getting the cover

We can get the Vertex Cover from the statevector outputted by the circuit. We'll choose the cover that we have the highest probability of sampling from the statevector as is done in qiskit's original notebook.

In [8]:
from qiskit.optimization.applications.ising.common import sample_most_likely

def get_size_cover(statevector: np.ndarray) -> int:
    return int(sum(
        vertex_cover.get_graph_solution(sample_most_likely(statevector))
    ))

## 5. Simulating the circuits

First we'll write a decorator that will decorate the simulations to print out useful information, namely (1) the time it took to run the simulation, (2) the expectation value that we get from the outputted statevector, and (3) the size of the resulting cover as determined by the `get_cover` function.

In [9]:
from typing import Callable
import time

f_type = Callable[[List[float]], np.ndarray]

def info_decorator(function: f_type) -> f_type:
    # `function` will be one of statevector_from_qiskit,
    # statevector_from_quasar, statevector_from_cirq, or statevector_from_vulcan.

    def f(params: List[float]) -> np.ndarray:
        print('='*40)
        print("Simulating with", function.__name__)
        print('-'*40)
        t0 = time.time()
        statevector = function(params)
        print("Time to completion : ", round(time.time() - t0, 2), "seconds")
        print("Expectation value  : ", round(expectation_value(statevector), 2))
        print("Size of cover      : ", get_size_cover(statevector))
        print('='*40, "\n")
        return statevector
        
    return f

We will be comparing a few different simulators. First will be qiskit's statevector simulator. *This runs completely locally.*

In [10]:
@info_decorator
def statevector_from_qiskit(params: List[float]) -> np.ndarray:
    return qiskit.execute(
        create_qiskit_circuit(params),
        qiskit.BasicAer.get_backend('statevector_simulator')
    ).result().get_statevector()

Next will be cirq's statevector simulator. *This runs completely locally.* Note that `cirq` by default simulates with single precision (ie `numpy.complex64`) whereas the other simulators by default simulate with double precison. So to be fair we will enforce that our cirq simulator simulates with double instead.

In [11]:
@info_decorator
def statevector_from_cirq(params: List[float]) -> np.ndarray:
    return cirq.Simulator(dtype=np.complex128).simulate(
        create_cirq_circuit(params)
    ).final_state

Next we'll use quasar's statevector simulator. *This runs completely locally.*

In [12]:
@info_decorator
def statevector_from_quasar(params: List[float]) -> np.ndarray:
    return quasar.QuasarSimulatorBackend().run_statevector(
        circuit=create_quasar_circuit(params)
    )

Finally, we'll use quasar's Vulcan GPU simulator. *This runs on forge servers.*

In [13]:
import qcware
from qcware.circuits.quasar_backend import QuasarBackend
qcware.config.set_api_key('Put your API key here!')

@info_decorator
def statevector_from_vulcan(params: List[float]) -> np.ndarray:
    return QuasarBackend("vulcan/simulator").run_statevector(
        circuit=create_quasar_circuit(params)
    )

## 6. Running and timing the circuits

Finally, we'll run the same circuit on each of the simulators and get all of the info from the `info_decorator`. *As a sanity check* we'll also make sure we get the same output probability distribution with the different simulators. *Note that the quasar, qiskit, and cirq statevectors will probably not be the same! They often differ by a global phase. Thus, their absolute value squared will be the same.* **This cell will take a long time to run!**

In [14]:
params = list(np.random.random(2*p) * np.pi)

qiskit_statevector = statevector_from_qiskit(params)
cirq_statevector   = statevector_from_cirq(params)
quasar_statevector = statevector_from_quasar(params)
vulcan_statevector = statev
ector_from_vulcan(params)

# check that probability vectors are the same
np.testing.assert_allclose(np.abs(qiskit_statevector)**2, np.abs(quasar_statevector)**2)
np.testing.assert_allclose(np.abs(cirq_statevector)**2,   np.abs(quasar_statevector)**2)
np.testing.assert_allclose(np.abs(vulcan_statevector)**2, np.abs(quasar_statevector)**2)

Simulating with statevector_from_qiskit
----------------------------------------
Time to completion :  707.0 seconds
Expectation value  :  248.82
Size of cover      :  20

Simulating with statevector_from_cirq
----------------------------------------
Time to completion :  211.18 seconds
Expectation value  :  248.82
Size of cover      :  20

Simulating with statevector_from_quasar
----------------------------------------
Time to completion :  1279.69 seconds
Expectation value  :  248.82
Size of cover      :  20

Simulating with statevector_from_vulcan
----------------------------------------
Time to completion :  58.72 seconds
Expectation value  :  248.82
Size of cover      :  20



As expected, the probability vectors are all the same (since the `np.testing.assert_allclose` call did not raise an exception). Similarly, we can see that the expectation values are also the same. We didn't do any optimization of the QAOA angles, so it's no surprise that the expectation value is so bad and that the size of the cover is completely wrong. For examples of optimizing the QAOA angles, please see some of the other notebooks on forge. The purpose of this notebook was simply to show that the GPU-accelerated Vulcan quantum simulator can dramatically reduce the amount of time it takes to simulate the execution of a large quantum circuit.

## 7. Outlook

The GPU simulator Vulcan significantly speeds up the circuit evaluation. Note that most of the bottleneck with the Vulcan simulator is actually just sending the statevector from the forge server back to the notebook. But if you run QAOA (optimize the angles, compute expectation values, etc.) with forge's QAOA solvers, then all the iteration is done on the forge servers and *no* time is spent sending the statevector across the wire!

[Back to top](#Running-a-Qiskit-QAOA-circuit-with-Quasar/Vulcan).